# Software Defect Prediction - Meta-Heuristic Feature Selection Pipeline
## NASA MDP D'' Dataset - Imbalanced Classification

**Author:** Complete Reproducible Experiment

**Strategy:**
- ✅ RepeatedStratifiedKFold (10x5)
- ✅ MCC as Primary Metric
- ✅ 3 Baseline Models (RF, XGB, CatBoost)
- ✅ Novel: Binary Grey Wolf Optimizer for Feature Selection
- ✅ Statistical Testing (Wilcoxon)
- ✅ Complete Reproducibility (fixed seeds)

In [ ]:
# ============================================================================
# IMPORTS AND DEPENDENCIES
# ============================================================================

import os
import glob
import warnings
import numpy as np
import pandas as pd
from scipy.io import arff
from scipy.stats import wilcoxon
from io import StringIO

# Machine Learning
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Imbalanced Learning - SMOTE EKLENDI!
from imblearn.over_sampling import SMOTE

# Metrics
from sklearn.metrics import (
    matthews_corrcoef, f1_score, recall_score, roc_auc_score,
    make_scorer, confusion_matrix
)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
sns.set_palette('husl')

# Fixed random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("[INFO] All dependencies loaded successfully!")
print(f"[INFO] Random seed: {RANDOM_SEED}")
print("[INFO] ✓ SMOTE (Synthetic Minority Over-sampling) imported!")

In [ ]:
# ============================================================================
# GOOGLE DRIVE MOUNTING
# ============================================================================

from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive mounted successfully!")

In [ ]:
# ============================================================================
# DATA LOADING
# ============================================================================

def load_arff_data(file_path):
    """Load ARFF file and convert to DataFrame"""
    try:
        data, meta = arff.loadarff(file_path)
        df = pd.DataFrame(data)
        
        # Decode byte strings
        for col in df.columns:
            if df[col].dtype == object:
                try:
                    df[col] = df[col].str.decode('utf-8')
                except AttributeError:
                    pass
        return df
    except Exception as e:
        print(f"[WARNING] scipy.io.arff failed: {e}")
        # Fallback: manual parsing
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            content = f.read()
        data_start = content.lower().find('@data')
        data_section = content[data_start + 5:].strip()
        df = pd.read_csv(StringIO(data_section), header=None)
        return df


def preprocess_dataset(df, target_col='defective'):
    """
    Preprocess dataset:
    - Separate features and target
    - Handle missing values
    - Convert target to binary (0/1)
    """
    # Assume last column is target
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    
    # Convert to numeric
    X = X.astype(np.float32)
    
    # Handle missing values (median imputation)
    if np.any(np.isnan(X)):
        col_median = np.nanmedian(X, axis=0)
        inds = np.where(np.isnan(X))
        X[inds] = np.take(col_median, inds[1])
    
    # Convert target to binary (0 = non-defective, 1 = defective)
    if y.dtype == object or y.dtype.name.startswith('str'):
        # Map 'true'/'false' or 'Y'/'N' to 1/0
        y_clean = []
        for val in y:
            if isinstance(val, bytes):
                val = val.decode('utf-8')
            val = str(val).strip().lower()
            if val in ['true', 'yes', 'y', '1']:
                y_clean.append(1)
            else:
                y_clean.append(0)
        y = np.array(y_clean, dtype=np.int32)
    else:
        y = y.astype(np.int32)
    
    return X, y


def load_datasets(dataset_dir):
    """
    Load all ARFF datasets from directory
    Returns: list of (dataset_name, X, y) tuples
    """
    arff_files = glob.glob(os.path.join(dataset_dir, '*.arff'))
    
    if not arff_files:
        raise FileNotFoundError(f"No ARFF files found in {dataset_dir}")
    
    datasets = []
    
    for file_path in arff_files:
        dataset_name = os.path.basename(file_path).replace('.arff', '')
        
        try:
            df = load_arff_data(file_path)
            X, y = preprocess_dataset(df)
            
            print(f"[LOADED] {dataset_name}: {X.shape[0]} samples, {X.shape[1]} features")
            print(f"         Class distribution: {np.bincount(y)}")
            
            datasets.append((dataset_name, X, y))
        except Exception as e:
            print(f"[ERROR] Failed to load {dataset_name}: {e}")
    
    return datasets


print("[INFO] Data loading functions ready!")

In [ ]:
# ============================================================================
# METRICS FUNCTION - F2 SCORE EKLENDI
# ============================================================================

def calculate_metrics(y_true, y_pred, y_pred_proba=None):
    """
    Calculate all required metrics
    
    Returns:
        dict: {MCC, F1, F2, Recall, AUC}
    """
    from sklearn.metrics import fbeta_score
    
    metrics = {}
    
    # MCC (Primary Metric)
    metrics['MCC'] = matthews_corrcoef(y_true, y_pred)
    
    # F1-Score
    metrics['F1'] = f1_score(y_true, y_pred, zero_division=0)
    
    # F2-Score (Recall-oriented: beta=2 gives recall 4x more weight than precision)
    metrics['F2'] = fbeta_score(y_true, y_pred, beta=2.0, zero_division=0)
    
    # Recall
    metrics['Recall'] = recall_score(y_true, y_pred, zero_division=0)
    
    # ROC-AUC (if probabilities provided)
    if y_pred_proba is not None:
        try:
            # Handle binary classification
            if len(np.unique(y_true)) > 1:
                metrics['AUC'] = roc_auc_score(y_true, y_pred_proba)
            else:
                metrics['AUC'] = 0.0
        except:
            metrics['AUC'] = 0.0
    else:
        metrics['AUC'] = 0.0
    
    return metrics


print("[INFO] Metrics function ready (with F2-Score)!")

In [ ]:
# ============================================================================
# BASELINE MODELS - İYİLEŞTİRİLMİŞ PARAMETRELER
# ============================================================================

def create_baseline_models(n_positive, n_negative):
    """
    Create baseline models with imbalance handling
    
    İYİLEŞTİRMELER:
    - Daha fazla estimator
    - Daha iyi hiperparametreler
    
    Args:
        n_positive: number of positive samples
        n_negative: number of negative samples
    
    Returns:
        dict: {model_name: model_instance}
    """
    # Calculate scale_pos_weight for XGBoost
    scale_pos_weight = n_negative / n_positive if n_positive > 0 else 1.0
    
    models = {
        'Random Forest': RandomForestClassifier(
            n_estimators=200,  # Increased from 100
            max_depth=15,      # Added depth control
            min_samples_split=5,
            min_samples_leaf=2,
            class_weight='balanced',
            random_state=RANDOM_SEED,
            n_jobs=-1
        ),
        
        'XGBoost': XGBClassifier(
            n_estimators=200,  # Increased from 100
            max_depth=6,       # Added depth control
            learning_rate=0.05, # Lower learning rate
            subsample=0.8,
            colsample_bytree=0.8,
            scale_pos_weight=scale_pos_weight,
            random_state=RANDOM_SEED,
            eval_metric='logloss',
            use_label_encoder=False
        ),
        
        'CatBoost': CatBoostClassifier(
            iterations=200,    # Increased from 100
            depth=6,           # Added depth control
            learning_rate=0.05,
            auto_class_weights='Balanced',
            random_seed=RANDOM_SEED,
            logging_level='Silent',
            allow_writing_files=False
        )
    }
    
    return models


print("[INFO] Baseline models defined (IMPROVED parameters)!")

In [ ]:
# ============================================================================
# BINARY GREY WOLF OPTIMIZER (BGWO)
# ============================================================================

class BinaryGreyWolfOptimizer:
    """
    Binary Grey Wolf Optimizer for Feature Selection
    
    Optimizes feature subset using GWO with sigmoid transfer function
    """
    
    def __init__(self, n_wolves=10, n_iterations=20, random_state=42):
        self.n_wolves = n_wolves
        self.n_iterations = n_iterations
        self.random_state = random_state
        np.random.seed(random_state)
        
        # Best solutions
        self.alpha_pos = None
        self.alpha_score = -float('inf')
        self.beta_pos = None
        self.beta_score = -float('inf')
        self.delta_pos = None
        self.delta_score = -float('inf')
        
        self.convergence_curve = []
    
    def sigmoid(self, x):
        """Sigmoid transfer function"""
        return 1.0 / (1.0 + np.exp(-10 * (x - 0.5)))
    
    def initialize_population(self, n_features):
        """Initialize binary population"""
        # Start with random feature subsets (30-70% features)
        population = np.random.rand(self.n_wolves, n_features) > 0.5
        
        # Ensure at least 3 features are selected in each wolf
        for i in range(self.n_wolves):
            if population[i].sum() < 3:
                indices = np.random.choice(n_features, 3, replace=False)
                population[i, indices] = True
        
        return population.astype(float)
    
    def optimize(self, X, y, fitness_func, verbose=True):
        """
        Run BGWO optimization
        
        Args:
            X: feature matrix
            y: target vector
            fitness_func: function(X_subset, y) -> score
            verbose: print progress
        
        Returns:
            best_features: boolean array of selected features
            best_score: fitness score
        """
        n_features = X.shape[1]
        
        # Initialize population
        positions = self.initialize_population(n_features)
        
        if verbose:
            print(f"\n[BGWO] Starting optimization...")
            print(f"       Wolves: {self.n_wolves}, Iterations: {self.n_iterations}")
        
        # Main optimization loop
        for iteration in range(self.n_iterations):
            # Evaluate fitness for each wolf
            for i in range(self.n_wolves):
                # Convert to binary mask
                feature_mask = positions[i] > 0.5
                
                # Ensure at least 3 features
                if feature_mask.sum() < 3:
                    indices = np.random.choice(n_features, 3, replace=False)
                    feature_mask[indices] = True
                    positions[i] = feature_mask.astype(float)
                
                # Calculate fitness
                try:
                    fitness = fitness_func(X[:, feature_mask], y)
                except:
                    fitness = 0.0
                
                # Update alpha, beta, delta
                if fitness > self.alpha_score:
                    self.delta_score = self.beta_score
                    self.delta_pos = self.beta_pos.copy() if self.beta_pos is not None else None
                    self.beta_score = self.alpha_score
                    self.beta_pos = self.alpha_pos.copy() if self.alpha_pos is not None else None
                    self.alpha_score = fitness
                    self.alpha_pos = positions[i].copy()
                elif fitness > self.beta_score:
                    self.delta_score = self.beta_score
                    self.delta_pos = self.beta_pos.copy() if self.beta_pos is not None else None
                    self.beta_score = fitness
                    self.beta_pos = positions[i].copy()
                elif fitness > self.delta_score:
                    self.delta_score = fitness
                    self.delta_pos = positions[i].copy()
            
            # Update coefficient a (linearly decreases from 2 to 0)
            a = 2.0 - iteration * (2.0 / self.n_iterations)
            
            # Update positions
            for i in range(self.n_wolves):
                for j in range(n_features):
                    # Calculate distance to alpha, beta, delta
                    r1, r2 = np.random.random(2)
                    A1 = 2 * a * r1 - a
                    C1 = 2 * r2
                    D_alpha = abs(C1 * self.alpha_pos[j] - positions[i, j])
                    X1 = self.alpha_pos[j] - A1 * D_alpha
                    
                    r1, r2 = np.random.random(2)
                    A2 = 2 * a * r1 - a
                    C2 = 2 * r2
                    D_beta = abs(C2 * self.beta_pos[j] - positions[i, j])
                    X2 = self.beta_pos[j] - A2 * D_beta
                    
                    r1, r2 = np.random.random(2)
                    A3 = 2 * a * r1 - a
                    C3 = 2 * r2
                    D_delta = abs(C3 * self.delta_pos[j] - positions[i, j])
                    X3 = self.delta_pos[j] - A3 * D_delta
                    
                    # Update position (average of X1, X2, X3)
                    new_pos = (X1 + X2 + X3) / 3.0
                    
                    # Apply sigmoid and convert to binary
                    positions[i, j] = self.sigmoid(new_pos)
            
            # Store convergence
            self.convergence_curve.append(self.alpha_score)
            
            if verbose and (iteration + 1) % 5 == 0:
                n_features_selected = (self.alpha_pos > 0.5).sum()
                print(f"       Iter {iteration + 1}/{self.n_iterations} | "
                      f"Best MCC: {self.alpha_score:.4f} | "
                      f"Features: {n_features_selected}/{n_features}")
        
        # Final best solution
        best_features = self.alpha_pos > 0.5
        
        if verbose:
            print(f"\n[BGWO] Optimization complete!")
            print(f"       Best MCC: {self.alpha_score:.4f}")
            print(f"       Selected: {best_features.sum()}/{n_features} features")
        
        return best_features, self.alpha_score


print("[INFO] Binary GWO implemented!")

In [ ]:
# ============================================================================
# META-HEURISTIC FEATURE SELECTION WRAPPER - İYİLEŞTİRİLMİŞ
# ============================================================================

class MetaHeuristicSelector:
    """
    Meta-Heuristic Feature Selection Wrapper for CatBoost
    
    Uses Binary GWO to find optimal feature subset,
    then trains CatBoost on selected features.
    
    İYİLEŞTİRMELER:
    - Daha güçlü BGWO parametreleri (12 wolves, 25 iterations)
    - Fitness: MCC + F2 kombinasyonu (recall-oriented)
    - Daha iyi model parametreleri
    """
    
    def __init__(self, n_wolves=12, n_iterations=25, random_state=42):
        self.n_wolves = n_wolves
        self.n_iterations = n_iterations
        self.random_state = random_state
        
        self.optimizer = None
        self.selected_features_ = None
        self.model_ = None
        self.scaler_ = None
    
    def _fitness_function(self, X_subset, y):
        """
        Fitness function: (MCC + F2) / 2
        Combines MCC with F2 for recall-oriented optimization
        """
        from sklearn.model_selection import StratifiedKFold
        from sklearn.metrics import fbeta_score
        
        # Quick 3-fold CV for speed
        cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=self.random_state)
        
        mcc_scores = []
        f2_scores = []
        
        for train_idx, val_idx in cv.split(X_subset, y):
            X_train, X_val = X_subset[train_idx], X_subset[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]
            
            # Train CatBoost with better parameters
            model = CatBoostClassifier(
                iterations=100,  # Increased from 50
                depth=6,         # Added depth control
                learning_rate=0.1,
                auto_class_weights='Balanced',
                random_seed=self.random_state,
                logging_level='Silent',
                allow_writing_files=False
            )
            
            model.fit(X_train, y_train, verbose=False)
            y_pred = model.predict(X_val)
            
            # Calculate MCC and F2
            mcc = matthews_corrcoef(y_val, y_pred)
            f2 = fbeta_score(y_val, y_pred, beta=2.0, zero_division=0)
            
            mcc_scores.append(mcc)
            f2_scores.append(f2)
        
        # Combined fitness: 60% MCC + 40% F2 (recall-oriented)
        avg_mcc = np.mean(mcc_scores)
        avg_f2 = np.mean(f2_scores)
        
        return 0.6 * avg_mcc + 0.4 * avg_f2
    
    def fit(self, X, y, verbose=True):
        """
        Fit the selector:
        1. Run BGWO to find best features
        2. Train final CatBoost on selected features
        """
        # Standardize features
        self.scaler_ = StandardScaler()
        X_scaled = self.scaler_.fit_transform(X)
        
        # Run BGWO with improved parameters
        self.optimizer = BinaryGreyWolfOptimizer(
            n_wolves=self.n_wolves,
            n_iterations=self.n_iterations,
            random_state=self.random_state
        )
        
        self.selected_features_, best_score = self.optimizer.optimize(
            X_scaled, y,
            fitness_func=self._fitness_function,
            verbose=verbose
        )
        
        # Train final model on selected features with better parameters
        X_selected = X_scaled[:, self.selected_features_]
        
        self.model_ = CatBoostClassifier(
            iterations=200,  # Increased from 100
            depth=6,
            learning_rate=0.05,  # Lower learning rate for better convergence
            auto_class_weights='Balanced',
            random_seed=self.random_state,
            logging_level='Silent',
            allow_writing_files=False
        )
        
        self.model_.fit(X_selected, y, verbose=False)
        
        return self
    
    def predict(self, X):
        """Predict using selected features"""
        X_scaled = self.scaler_.transform(X)
        X_selected = X_scaled[:, self.selected_features_]
        return self.model_.predict(X_selected)
    
    def predict_proba(self, X):
        """Predict probabilities using selected features"""
        X_scaled = self.scaler_.transform(X)
        X_selected = X_scaled[:, self.selected_features_]
        return self.model_.predict_proba(X_selected)[:, 1]


print("[INFO] MetaHeuristicSelector implemented (IMPROVED: 12 wolves, 25 iters, MCC+F2 fitness)!")

In [ ]:
# ============================================================================
# EXPERIMENT EXECUTION - F2 SCORE + SMOTE EKLENDI
# ============================================================================

def run_experiment(dataset_name, X, y, n_splits=10, n_repeats=5, use_smote=True):
    """
    Run complete experiment:
    - RepeatedStratifiedKFold CV
    - SMOTE on training set (each fold)
    - 3 Baselines + 1 Novel Approach
    - Store all metrics (including F2)
    
    Args:
        dataset_name: name of the dataset
        X: feature matrix
        y: target vector
        n_splits: number of CV splits
        n_repeats: number of CV repeats
        use_smote: whether to apply SMOTE (default: True)
    
    Returns:
        DataFrame with all results
    """
    print("\n" + "="*70)
    print(f"DATASET: {dataset_name}")
    print("="*70)
    print(f"Samples: {X.shape[0]}, Features: {X.shape[1]}")
    print(f"Class distribution: {np.bincount(y)}")
    
    if use_smote:
        print("🔥 SMOTE ENABLED: Synthetic oversampling will be applied to training sets")
    
    # Calculate class weights
    n_positive = np.sum(y == 1)
    n_negative = np.sum(y == 0)
    
    # Create baseline models
    baseline_models = create_baseline_models(n_positive, n_negative)
    
    # Add novel approach (with improved parameters)
    all_models = baseline_models.copy()
    all_models['BGWO-CatBoost'] = MetaHeuristicSelector(
        n_wolves=12,      # Improved from 8
        n_iterations=25,  # Improved from 15
        random_state=RANDOM_SEED
    )
    
    # Setup CV
    cv = RepeatedStratifiedKFold(
        n_splits=n_splits,
        n_repeats=n_repeats,
        random_state=RANDOM_SEED
    )
    
    # Store results
    results = []
    
    # Standardize features (for baseline models)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Run CV
    fold_idx = 0
    total_folds = n_splits * n_repeats
    
    print(f"\nRunning {total_folds} folds ({n_splits} splits × {n_repeats} repeats)...")
    print("-" * 70)
    
    for train_idx, test_idx in cv.split(X, y):
        fold_idx += 1
        
        # Split data
        X_train_raw, X_test_raw = X[train_idx], X[test_idx]
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        # APPLY SMOTE TO TRAINING SET ONLY (avoid data leakage!)
        if use_smote:
            # Check if we have enough minority samples for SMOTE
            min_samples = min(np.bincount(y_train))
            
            if min_samples >= 2:  # SMOTE needs at least 2 minority samples
                try:
                    # For raw data (MetaHeuristic)
                    smote_raw = SMOTE(random_state=RANDOM_SEED, k_neighbors=min(5, min_samples-1))
                    X_train_raw_smote, y_train_smote = smote_raw.fit_resample(X_train_raw, y_train)
                    
                    # For scaled data (Baselines)
                    smote_scaled = SMOTE(random_state=RANDOM_SEED, k_neighbors=min(5, min_samples-1))
                    X_train_smote, _ = smote_scaled.fit_resample(X_train, y_train)
                    
                    # Show SMOTE effect on first fold
                    if fold_idx == 1:
                        print(f"   [SMOTE] Before: {np.bincount(y_train)} → After: {np.bincount(y_train_smote)}")
                    
                    # Use SMOTE'd data
                    X_train_raw = X_train_raw_smote
                    X_train = X_train_smote
                    y_train = y_train_smote
                    
                except Exception as e:
                    if fold_idx == 1:
                        print(f"   [SMOTE] Warning: Could not apply SMOTE on fold {fold_idx}: {e}")
            else:
                if fold_idx == 1:
                    print(f"   [SMOTE] Skipped: Not enough minority samples ({min_samples})")
        
        # Show progress every 10 folds
        if fold_idx % 10 == 0 or fold_idx == 1:
            print(f"Fold {fold_idx}/{total_folds}...")
        
        # Test each model
        for model_name, model in all_models.items():
            try:
                # For MetaHeuristic, use raw data (it has its own scaler)
                if model_name == 'BGWO-CatBoost':
                    # Only show verbose for first fold
                    verbose = (fold_idx == 1)
                    model.fit(X_train_raw, y_train, verbose=verbose)
                    y_pred = model.predict(X_test_raw)
                    y_pred_proba = model.predict_proba(X_test_raw)
                else:
                    # Baseline models use scaled data
                    model.fit(X_train, y_train)
                    y_pred = model.predict(X_test)
                    
                    # Get probabilities
                    if hasattr(model, 'predict_proba'):
                        y_pred_proba = model.predict_proba(X_test)[:, 1]
                    else:
                        y_pred_proba = y_pred
                
                # Calculate metrics (includes F2 now)
                metrics = calculate_metrics(y_test, y_pred, y_pred_proba)
                
                # Store results
                result_row = {
                    'Dataset': dataset_name,
                    'Model': model_name,
                    'Fold': fold_idx,
                    'MCC': metrics['MCC'],
                    'F1': metrics['F1'],
                    'F2': metrics['F2'],  # NEW!
                    'Recall': metrics['Recall'],
                    'AUC': metrics['AUC']
                }
                results.append(result_row)
                
            except Exception as e:
                print(f"[ERROR] {model_name} failed on fold {fold_idx}: {e}")
    
    print("-" * 70)
    print(f"✓ Completed {total_folds} folds for {len(all_models)} models")
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df


print("[INFO] Experiment function ready (with F2 score + SMOTE)!")

In [ ]:
# ============================================================================
# REPORTING & VISUALIZATION
# ============================================================================

def generate_report(results_df):
    """
    Generate summary statistics and visualizations
    """
    print("\n" + "="*70)
    print("SUMMARY STATISTICS")
    print("="*70)
    
    # Calculate mean and std for each model
    summary = results_df.groupby('Model')[['MCC', 'F1', 'Recall', 'AUC']].agg(['mean', 'std'])
    
    # Flatten column names
    summary.columns = ['_'.join(col).strip() for col in summary.columns.values]
    
    # Sort by MCC (primary metric)
    summary = summary.sort_values('MCC_mean', ascending=False)
    
    print("\n" + summary.to_string())
    
    # Boxplot
    print("\n" + "="*70)
    print("GENERATING BOXPLOT...")
    print("="*70)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Boxplot for MCC scores
    results_df.boxplot(column='MCC', by='Model', ax=ax, patch_artist=True)
    
    ax.set_xlabel('Model', fontsize=12, fontweight='bold')
    ax.set_ylabel('MCC Score', fontsize=12, fontweight='bold')
    ax.set_title('MCC Score Comparison (Primary Metric)', fontsize=14, fontweight='bold')
    plt.suptitle('')  # Remove default title
    
    ax.grid(axis='y', alpha=0.3)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    plt.savefig('mcc_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✓ Boxplot saved: mcc_comparison.png")
    
    return summary


def statistical_test(results_df, baseline_model, novel_model):
    """
    Perform Wilcoxon Signed-Rank Test
    
    Tests if novel approach is significantly better than baseline
    """
    print("\n" + "="*70)
    print("STATISTICAL SIGNIFICANCE TEST")
    print("="*70)
    print(f"Baseline: {baseline_model}")
    print(f"Novel:    {novel_model}")
    print("-" * 70)
    
    # Get MCC scores
    baseline_scores = results_df[results_df['Model'] == baseline_model]['MCC'].values
    novel_scores = results_df[results_df['Model'] == novel_model]['MCC'].values
    
    # Wilcoxon test
    statistic, p_value = wilcoxon(novel_scores, baseline_scores, alternative='greater')
    
    print(f"\nWilcoxon Signed-Rank Test:")
    print(f"  Statistic: {statistic:.4f}")
    print(f"  P-value:   {p_value:.6f}")
    
    if p_value < 0.05:
        print(f"\n✓ SIGNIFICANT: {novel_model} is significantly better (p < 0.05)")
    else:
        print(f"\n✗ NOT SIGNIFICANT: No significant difference (p >= 0.05)")
    
    print("="*70)
    
    return statistic, p_value


print("[INFO] Reporting functions ready!")

In [ ]:
# ============================================================================
# MAIN EXECUTION - HER DATASET AYRI AYRI İŞLENİR + F2 SCORE + SMOTE
# ============================================================================

# Set dataset directory
DATASET_DIR = '/content/drive/MyDrive/nasa-defect-gwo-kan/dataset'

print("\n" + "#"*70)
print("# SOFTWARE DEFECT PREDICTION - META-HEURISTIC PIPELINE")
print("# NASA MDP D'' Dataset - Imbalanced Classification")
print("# HER DATASET İÇİN AYRI AYRI SONUÇLAR + F2 SCORE + SMOTE")
print("# İYİLEŞTİRMELER: 12 wolves, 25 iters, MCC+F2 fitness, SMOTE")
print("#"*70)

# Load datasets
print("\n[STEP 1] Loading datasets...")
datasets = load_datasets(DATASET_DIR)

print(f"\n[INFO] Loaded {len(datasets)} datasets")
print(f"[INFO] Her dataset için ayrı ayrı işlem yapılacak ve sonuçlar gösterilecek")
print(f"[INFO] 🔥 SMOTE aktif - Training set'lere synthetic oversampling uygulanacak!")

# Store all results for final summary
all_results = []
dataset_summaries = []

# Run experiments on each dataset SEPARATELY
for idx, (dataset_name, X, y) in enumerate(datasets, 1):
    print("\n" + "█"*70)
    print(f"█ DATASET {idx}/{len(datasets)}: {dataset_name}")
    print("█"*70)

    # Skip very small datasets
    if X.shape[0] < 100:
        print(f"\n[SKIP] {dataset_name}: Too small ({X.shape[0]} samples)")
        print("█"*70 + "\n")
        continue

    # Run experiment for this dataset
    print(f"\n[RUNNING] Experiment başlatıldı: {dataset_name}")
    results_df = run_experiment(dataset_name, X, y, n_splits=10, n_repeats=5, use_smote=True)
    all_results.append(results_df)

    # IMMEDIATELY show results for THIS dataset
    print("\n" + "▼"*70)
    print(f"▼ SONUÇLAR: {dataset_name}")
    print("▼"*70)

    # Calculate summary for this dataset (including F2)
    summary = results_df.groupby('Model')[['MCC', 'F1', 'F2', 'Recall', 'AUC']].agg(['mean', 'std'])
    summary.columns = ['_'.join(col).strip() for col in summary.columns.values]
    summary = summary.sort_values('MCC_mean', ascending=False)

    # Store for final comparison
    dataset_summaries.append({
        'dataset': dataset_name,
        'summary': summary
    })

    # Print results
    print("\n📊 ÖZET İSTATİSTİKLER:")
    print(summary.to_string())

    # Find best model for this dataset
    best_model = summary.index[0]
    best_mcc = summary.loc[best_model, 'MCC_mean']
    best_f2 = summary.loc[best_model, 'F2_mean']

    print(f"\n🏆 EN İYİ MODEL: {best_model}")
    print(f"   MCC (mean ± std): {best_mcc:.4f} ± {summary.loc[best_model, 'MCC_std']:.4f}")
    print(f"   F2  (mean ± std): {best_f2:.4f} ± {summary.loc[best_model, 'F2_std']:.4f}")

    # Show how BGWO-CatBoost performed
    if 'BGWO-CatBoost' in summary.index:
        bgwo_mcc = summary.loc['BGWO-CatBoost', 'MCC_mean']
        bgwo_mcc_std = summary.loc['BGWO-CatBoost', 'MCC_std']
        bgwo_f2 = summary.loc['BGWO-CatBoost', 'F2_mean']
        bgwo_f2_std = summary.loc['BGWO-CatBoost', 'F2_std']
        bgwo_recall = summary.loc['BGWO-CatBoost', 'Recall_mean']
        bgwo_rank = list(summary.index).index('BGWO-CatBoost') + 1

        print(f"\n🔬 BGWO-CatBoost (Novel Method - IMPROVED + SMOTE):")
        print(f"   MCC    (mean ± std): {bgwo_mcc:.4f} ± {bgwo_mcc_std:.4f}")
        print(f"   F2     (mean ± std): {bgwo_f2:.4f} ± {bgwo_f2_std:.4f}")
        print(f"   Recall (mean ± std): {bgwo_recall:.4f} ± {summary.loc['BGWO-CatBoost', 'Recall_std']:.4f}")
        print(f"   Sıralama: {bgwo_rank}/{len(summary)} model arasında")

    print("\n" + "█"*70)
    print(f"█ ✅ TAMAMLANDI: {dataset_name} ({idx}/{len(datasets)})")
    print("█"*70 + "\n\n")

# ============================================================================
# FINAL SUMMARY - ALL DATASETS COMBINED
# ============================================================================

if len(all_results) > 0:
    print("\n" + "🎯"*35)
    print("🎯 TÜM DATASET'LER İÇİN GENEL ÖZET")
    print("🎯"*35)

    # Combine all results
    final_results = pd.concat(all_results, ignore_index=True)

    # Overall statistics
    print("\n📊 GENEL İSTATİSTİKLER (Tüm Dataset'ler Birleştirildi):")
    print("="*70)

    overall_summary = final_results.groupby('Model')[['MCC', 'F1', 'F2', 'Recall', 'AUC']].agg(['mean', 'std'])
    overall_summary.columns = ['_'.join(col).strip() for col in overall_summary.columns.values]
    overall_summary = overall_summary.sort_values('MCC_mean', ascending=False)

    print(overall_summary.to_string())

    # Statistical test (compare best baseline vs novel)
    print("\n" + "="*70)
    print("📈 İSTATİSTİKSEL TEST")
    print("="*70)

    # Find best baseline (highest mean MCC)
    baseline_models = ['Random Forest', 'XGBoost', 'CatBoost']
    baseline_means = {}
    for model in baseline_models:
        mean_mcc = final_results[final_results['Model'] == model]['MCC'].mean()
        baseline_means[model] = mean_mcc

    best_baseline = max(baseline_means, key=baseline_means.get)

    print(f"\nEn İyi Baseline: {best_baseline} (MCC: {baseline_means[best_baseline]:.4f})")

    # Test against novel approach
    if 'BGWO-CatBoost' in final_results['Model'].values:
        stat, p_val = statistical_test(final_results, best_baseline, 'BGWO-CatBoost')

    # Generate overall boxplot
    print("\n" + "="*70)
    print("📊 GENEL BOXPLOT OLUŞTURULUYOR...")
    print("="*70)

    fig, ax = plt.subplots(figsize=(12, 6))
    final_results.boxplot(column='MCC', by='Model', ax=ax, patch_artist=True)
    ax.set_xlabel('Model', fontsize=12, fontweight='bold')
    ax.set_ylabel('MCC Score', fontsize=12, fontweight='bold')
    ax.set_title('MCC Score Comparison - All Datasets (IMPROVED + SMOTE)', fontsize=14, fontweight='bold')
    plt.suptitle('')
    ax.grid(axis='y', alpha=0.3)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('mcc_comparison_all_datasets_smote.png', dpi=300, bbox_inches='tight')
    plt.show()

    print("✓ Boxplot kaydedildi: mcc_comparison_all_datasets_smote.png")

    # F2 boxplot
    fig, ax = plt.subplots(figsize=(12, 6))
    final_results.boxplot(column='F2', by='Model', ax=ax, patch_artist=True)
    ax.set_xlabel('Model', fontsize=12, fontweight='bold')
    ax.set_ylabel('F2 Score (Recall-oriented)', fontsize=12, fontweight='bold')
    ax.set_title('F2 Score Comparison - All Datasets (with SMOTE)', fontsize=14, fontweight='bold')
    plt.suptitle('')
    ax.grid(axis='y', alpha=0.3)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('f2_comparison_all_datasets_smote.png', dpi=300, bbox_inches='tight')
    plt.show()

    print("✓ F2 Boxplot kaydedildi: f2_comparison_all_datasets_smote.png")

    # Save results
    print("\n" + "="*70)
    print("💾 SONUÇLAR KAYDEDİLİYOR...")
    print("="*70)

    final_results.to_csv('sdp_results_detailed_smote.csv', index=False)
    print("✓ Detaylı sonuçlar: sdp_results_detailed_smote.csv")

    overall_summary.to_csv('sdp_results_summary_smote.csv')
    print("✓ Genel özet: sdp_results_summary_smote.csv")

    # Save individual dataset summaries
    with open('sdp_results_by_dataset_smote.txt', 'w') as f:
        f.write("="*70 + "\n")
        f.write("DATASET'E GÖRE SONUÇLAR (WITH F2 + SMOTE)\n")
        f.write("="*70 + "\n\n")

        for item in dataset_summaries:
            f.write(f"\n{'='*70}\n")
            f.write(f"Dataset: {item['dataset']}\n")
            f.write(f"{'='*70}\n")
            f.write(item['summary'].to_string())
            f.write("\n\n")

    print("✓ Dataset bazlı özet: sdp_results_by_dataset_smote.txt")

    print("\n" + "🎯"*35)
    print("🎯 TÜM DENEYLER TAMAMLANDI!")
    print("🎯"*35)

    print(f"\n✅ İşlenen Dataset Sayısı: {len(all_results)}")
    print(f"✅ Toplam Fold Sayısı: {len(final_results)}")
    print(f"✅ Test Edilen Model Sayısı: {len(overall_summary)}")

    # Show improvement summary
    print("\n" + "🚀"*35)
    print("🚀 İYİLEŞTİRMELER:")
    print("🚀"*35)
    print("✓ BGWO: 8→12 wolves, 15→25 iterations")
    print("✓ Fitness: Pure MCC → 60% MCC + 40% F2 (recall-oriented)")
    print("✓ Models: 100→200 estimators/iterations")
    print("✓ Metrics: MCC, F1, F2, Recall, AUC")
    print("✓ Hyperparameters: depth control, learning rate optimization")
    print("✓ 🔥 SMOTE: Synthetic minority oversampling on each training fold")
    print("🚀"*35)

else:
    print("\n[ERROR] Hiçbir dataset işlenemedi!")